In [ ]:
pip install torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [ ]:
# Define transformations for the training set, add data augmentation or normalization here
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11416895.96it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 347766.29it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3196827.25it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3882316.85it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)  # Output: (32, 26, 26)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)  # Output: (64, 24, 24)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)  # Adjust the input features here
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)  # Output: (32, 13, 13)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)  # Output: (64, 6, 6)
        x = x.view(-1, 64 * 5 * 5)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
import torch.optim as optim
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the training and test datasets
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

# Define data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Initialize the model, optimizer, and loss function
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [ ]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Train the model for several epochs
for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, criterion, epoch)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.331074
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.204851
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.098228
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.149701
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.086318
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.163080
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.037712
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.014779
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.009493
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.008883
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.048643
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.030548
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.083290
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.013666
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.043076
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.049077
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.021826
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.057680
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.022303
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.089443
Train Epoch:

In [ ]:
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')

# Evaluate the model
test(model, device, test_loader, criterion)



Test set: Average loss: 0.0023, Accuracy: 404/10000 (4.04%)



In [ ]:
torch.save(model.state_dict(), "mnist_cnn.pth")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the path in Google Drive
model_save_path = "/content/drive/My Drive/mnist_cnn.pth"

# Save the model
torch.save(model.state_dict(), model_save_path)


In [ ]:
!ls /content/drive/My\ Drive


 17.jpg   badge.jpg	     error.jpg		  mnist_cnn.pth
 2.jpg	  Classroom	    'ishika resume.pdf'  'Project proposal.gdoc'
 7.jpg	 'Colab Notebooks'   IshikaResume.pdf


In [ ]:
model = Net()
model.load_state_dict(torch.load("/content/drive/My Drive/mnist_cnn.pth"))
model.eval()


Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((28, 28)),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    return image

In [ ]:
from PIL import Image
# Path to the image in Google Drive
image_path = "/content/drive/My Drive/2.jpg"  # Update this path if necessary
input_image = preprocess_image(image_path)

# Move input to the same device as the model
input_image = input_image.to(device)

# Get the prediction
with torch.no_grad():
    output = model(input_image)
    predicted_class = output.argmax(dim=1, keepdim=True)

print(f"Predicted class: {predicted_class.item()}")

NameError: name 'preprocess_image' is not defined

In [ ]:
# Add these print statements before the prediction
print("Input image tensor shape:", input_image.shape)
print("Input image tensor values:", input_image)

# Get the prediction
with torch.no_grad():
    output = model(input_image)
    print("Raw prediction scores:", output)
    predicted_class = output.argmax(dim=1, keepdim=True)


Input image tensor shape: torch.Size([1, 1, 28, 28])
Input image tensor values: tensor([[[[ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
            2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
            2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
            2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
          [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,
            2.8088,  2.8088,  2.7960,  2.8215,  2.8215,  2.8215,  2.8088,
            2.8088,  2.7960,  2.7960,  2.7960,  2.8215,  2.8215,  2.8215,
            2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
          [ 2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.6942,  2.4651,
            2.1214,  1.3450,  0.9250,  1.0650,  1.1414,  1.0904,  1.0523,
            0.9504,  0.8232,  1.0268,  0.8741,  1.2432,  2.0323,  2.7324,
            2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215,  2.8215],
          [ 2.8215,  2.8215, 